In [79]:
import math
from scipy.special import comb
def Model_Type(lam, mu, System_Capacity, Num_S, Population_z, Service_t, Arrival_t):
    model_type = ""
    p0 = None
    pk = []
    avg_num_in_sys = 0
    avg_num_in_q = 0
    avg_time_in_sys = 0
    avg_time_in_queue = 0
    rho = 0  # set a default value for rho

    #-------------------------M/M/1 MODEL----------------------------------------

    if System_Capacity == float('inf') and Num_S == 1 and Service_t == 'Exponential' and Arrival_t == 'poisson':
        model_type = 'M/M/1'
        rho = lam/mu
        p0 = 1 - rho
        pk = [(rho ** i) * p0 for i in range(Population_z)]
        avg_num_in_sys = rho / (1 - rho)
        avg_num_in_q  = rho/(mu-lam)
        avg_time_in_sys = 1 / (mu - lam)
        avg_time_in_queue = rho / (mu - lam)
        return ('M/M/1', lam, mu,rho, p0, pk, avg_num_in_sys,avg_num_in_q, avg_time_in_sys, avg_time_in_queue)

    #-------------------------M/M/k/1 MODEL----------------------------------------

    if System_Capacity == Population_z and Num_S == 1 and Service_t == 'Exponential' and Arrival_t == 'poisson':
        model_type = 'M/M/1/K'
        rho = lam / (mu * Num_S)

        # Calculate effective arrival rate and service rate
        if rho == 1:
            effective_arrival_rate = lam / Num_S
            mu = mu / Num_S
        else:
            effective_arrival_rate = lam * (1 - rho)

        # Calculate probability of zero customers in the system
        p0 = 1 - rho
        # Calculate average number of customers in the system
        if rho == 1:
            avg_num_in_q = Population_z ** 2 / (2 * (Population_z - Num_S))
        else:
            avg_num_in_q = (rho ** 2) / (1 - rho) * ((Population_z + 1) - ((Population_z + 1) * ((1 - rho) ** Num_S)) + (Num_S * (rho ** Num_S) * (1 - rho)))
        avg_num_in_sys = avg_num_in_q + Population_z

        # Calculate average waiting time in queue and time spent in system
        avg_time_in_queue = avg_num_in_q / effective_arrival_rate
        avg_time_in_sys = avg_num_in_sys / effective_arrival_rate

        W = avg_time_in_queue + 1/mu

        # Calculate probability of n customers in the system, for n > 0 and n <= K-1 
        Pn_list=[]
        for n in range(1, System_Capacity):
            if n <= Population_z:
                Pn = (rho ** n) / math.factorial(n) * p0
            else:
                Pn = (rho ** n) / (math.factorial(Num_S) * (Population_z ** (n - Population_z))) * p0
            Pn_list.append(Pn)
        return ('M/M/1/k', lam, mu,rho, p0, Pn_list, avg_num_in_sys,avg_num_in_q, avg_time_in_sys, avg_time_in_queue)
    
        if p0 is None:
            raise ValueError("Invalid input parameters")
        
 #-------------------------M/M/C MODEL----------------------------------------

    if System_Capacity < float('inf') and Num_S > 1 and Service_t == 'Exponential' and Arrival_t == 'poisson':
        # Calculate utilization
        #rho = lam / (mu * Num_S)
        rho = lam / (Num_S * mu)

        # Calculate effective arrival rate and service rate
        if rho == 1:
            effective_arrival_rate = lam / Num_S
            mu = mu / Num_S
        else:
            effective_arrival_rate = lam * (1 - rho)

        # Calculate probability of zero customers in the system
        if rho == 1:
            p0 = 0
        else:
            p0 = 1 / (sum([(Num_S * rho) ** n / comb(Num_S, n) for n in range(Num_S)]) + ((Num_S * rho) ** Num_S / comb(System_Capacity, Num_S)) * (1 / (1 - rho)))

        # Calculate average number of customers in the system
        if rho == 1:
            avg_num_in_q = Population_z ** 2 / (2 * (Population_z - Num_S))
        else:
            #avg_num_in_q = (rho ** 2) / (1 - rho) * ((Population_z + 1) - ((Population_z + 1) * ((1 - rho) ** (Num_S + 1))) + (Num_S * (rho ** Num_S) * (1 - rho)))
            avg_num_in_q = rho**2 / (Num_S*(1-rho/Num_S))


        avg_num_in_sys  = Num_S*rho + rho*(rho/Num_S / (1-rho/Num_S))
        
        # Calculate average waiting time in queue and time spent in system
        #avg_time_in_queue = avg_num_in_q / lam
        avg_time_in_queue = avg_num_in_q / lam
        #avg_time_in_sys = avg_time_in_queue + 1 / mu
        avg_time_in_sys = avg_num_in_sys / lam

        # Calculate probability of n customers in the system, for n > 0 and n <= K-1 
        Pn_list = []
        for n in range(1, System_Capacity):
            if n <= Population_z:
                Pn = (rho ** n) / math.factorial(n) * p0
            else:
                Pn = (rho ** n) / (math.factorial(Num_S) * (Population_z ** (n - Population_z))) * p0
            Pn_list.append(Pn)                                        
          
        # Calculate probability that the system is full
        if rho == 1:
            pk = 1 / (Num_S + 1)
        else:
            pk = ((rho ** Population_z) / math.factorial(Num_S)) * ((Num_S * (1 - rho)) ** (-Population_z))

        # Return performance measures as a tuple
        return ('M/M/C', lam, mu, p0, pk,Pn_list, avg_num_in_sys, avg_time_in_sys, avg_time_in_queue)
 #-------------------------M/M/∞MODEL----------------------------------------
    if System_Capacity==float('inf') and Num_S>1 and Service_t == 'Exponential'and Arrival_t =='poisson':
                                   
         # Calculate utilization
        rho = lam / mu

        # Calculate probability of zero customers in the system
        p0 = 1 - rho

        # Calculate average number of customers in the system and in queue
        avg_num_in_q = 0
        avg_num_in_sys = lam/mu

        # Calculate average waiting time in queue and time spent in system
        avg_time_in_queue = 0
        avg_time_in_sys = 1/mu
        return ('M/M/∞', lam, mu,rho, p0,  avg_num_in_sys,avg_num_in_q, avg_time_in_sys, avg_time_in_queue)



In [53]:
Model_Type(3, 4, 10 , 1, 10, "Exponential", "poisson")


('M/M/1/k',
 3,
 4,
 0.75,
 0.25,
 [0.1875,
  0.0703125,
  0.017578125,
  0.0032958984375,
  0.000494384765625,
  6.1798095703125e-05,
  6.621224539620536e-06,
  6.207398005894252e-07,
  5.172831671578544e-08],
 28.984375,
 18.984375,
 38.645833333333336,
 25.3125)

In [55]:
Model_Type(10, 5, float('inf'), 2, 100, "Exponential", "poisson")

('M/M/∞', 10, 5, 2.0, -1.0, 2.0, 0, 0.2, 0)

In [80]:
Model_Type(4, 2, 5, 2, 3, "Exponential", "poisson")

('M/M/C',
 4,
 1.0,
 0,
 0.3333333333333333,
 [0.0, 0.0, 0.0, 0.0],
 3.0,
 0.75,
 1.125)